In [88]:
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score, roc_auc_score, recall_score, precision_score
from sklearn.metrics import confusion_matrix

def sigmoid(x):
    """
    Compute the sigmoid function for the input array.
    
    Parameters:
    x (array-like): Input array or scalar
    
    Returns:
    array-like: Sigmoid of the input
    """
    return 1 / (1 + np.exp(-x))

def find_best_threshold(df):
    """
    Find the best threshold for binary classification based on F1 score.
    
    Parameters:
    df (DataFrame): DataFrame containing prediction and label columns
    
    Returns:
    float: Best threshold value for the given predictions and labels
    """
    thresholds = np.linspace(-3, 3, 1000)
    predictions, labels = df.prediction, df.label
    f1_scores = [f1_score(labels, predictions > threshold) for threshold in thresholds]
    best_threshold = thresholds[np.argmax(f1_scores)]
    return best_threshold




def calculate_metrics_at_threshold(threshold):
    """
    Calculate evaluation metrics for a given threshold.
    
    Parameters:
    threshold (float): Threshold value for binary classification
    
    Returns:
    function: Function that calculates metrics for the given DataFrame
    """
    def metrics_function(df):
        predictions, labels = df.prediction, df.label
        auc = roc_auc_score(labels, predictions)
        binary_predictions = predictions > threshold
        #recall = recall_score(labels, binary_predictions)
        #precision = precision_score(labels, binary_predictions)
        f1 = f1_score(labels, binary_predictions)

        tn, fp, fn, tp = confusion_matrix(labels, binary_predictions).ravel()
        precision = tp / (tp+fp) # PPV
        recall = tp / (tp+fn) # NPV
        specificity = tn / (tn + fp)
        sensitivity = tp / (tp + fn)
        
        return {
            "auc": auc,
            "f1": f1,
            "precision": precision,
            "recall": recall,
            "specificity": specificity,
            "sensitivity": sensitivity,
        }
    return metrics_function

def evaluate_metrics(df):
    """
    Evaluate metrics for train and test splits in the DataFrame.
    
    Parameters:
    df (DataFrame): DataFrame containing prediction, label, and split columns
    
    Returns:
    DataFrame: DataFrame containing evaluation metrics for each split
    """
    #df = df.copy()
    #df["prediction"] = sigmoid(df.prediction)
    #train_df = df.query("split == 'train'")
    best_threshold = 0 # find_best_threshold(train_df)
    grouped_metrics = df.groupby("split").apply(calculate_metrics_at_threshold(best_threshold), include_groups=False)
    return grouped_metrics.apply(pd.Series)



In [89]:
import pandas as pd
import duckdb

In [90]:
experiments = pd.read_parquet("gs://bucket_name/iteration_n/data2/experiments.parquet")

In [91]:
prediction = pd.read_parquet("gs://bucket_name/iteration_n/data2/prediction")

In [363]:
duckdb.sql("""
--count of experiments per marker
select marker, count(distinct(e.experiment_uuid))
from prediction p
join experiments e on e.experiment_uuid = p.experiment_uuid
where p.split in ('holdout', 'test')
group by marker
order by marker
""").df()

,marker,count(DISTINCT e.experiment_uuid)
0,EGFR,120
1,KEAP1,120
2,KRAS,120
3,STK11,120
4,TP53,120


In [364]:
sub = duckdb.sql("""
select 
    marker,
    split
    train_cohort, 
    test_fold, 
    count(distinct(dev_fold)), 
    count(distinct(seed)), 
    count(distinct(e.experiment_uuid))
from prediction p 
join experiments e on e.experiment_uuid = p.experiment_uuid
where p.split in ('holdout', 'test')
group by marker, train_cohort, test_fold, split
order by marker, train_cohort, test_fold, split
""").df()
sub

,marker,train_cohort,test_fold,count(DISTINCT dev_fold),count(DISTINCT seed),count(DISTINCT e.experiment_uuid)
0,EGFR,holdout,0,4,3,12
1,EGFR,holdout,0,4,3,12
2,EGFR,holdout,1,4,3,12
3,EGFR,holdout,1,4,3,12
4,EGFR,holdout,2,4,3,12
...,...,...,...,...,...,...
95,TP53,test,2,4,3,12
96,TP53,test,3,4,3,12
97,TP53,test,3,4,3,12
98,TP53,test,4,4,3,12


In [365]:
ensemble = duckdb.sql("""
select 
    train_cohort, 
    marker, 
    test_fold, 
    wsi_uuid, 
    avg(prediction) as prediction, 
    avg(label) as label, 
    split
from prediction p 
join experiments e on e.experiment_uuid = p.experiment_uuid
where p.split in ('holdout', 'test')
group by
    wsi_uuid,
    split,
    marker, 
    test_fold, 
    train_cohort, 
    holdout_cohort,
""").df()
ensemble

,train_cohort,marker,test_fold,wsi_uuid,prediction,label,split
0,hlcc,KRAS,3,TCGA-49-4488-01Z-00-DX9.bf1e9d28-3ca2-4a62-bf3...,0.557541,0.0,holdout
1,hlcc,KRAS,3,TCGA-50-5068-01Z-00-DX2.0492A5C6-09CB-424B-BE2...,0.431438,0.0,holdout
2,hlcc,KRAS,3,TCGA-50-6593-01Z-00-DX1.a63e298c-cbe7-44e8-8d8...,0.395326,0.0,holdout
3,hlcc,KRAS,3,TCGA-55-A48Z-01Z-00-DX1.0867DC6A-2A51-4CF1-AE3...,0.518045,0.0,holdout
4,hlcc,KRAS,3,TCGA-86-8073-01Z-00-DX1.33c016fc-5c9e-4ad6-8de...,0.356221,0.0,holdout
...,...,...,...,...,...,...,...
84265,tcga,KEAP1,1,43f0f92e-364e-4cb1-b037-2d20660987d0_73ddb7ea-...,-1.780631,0.0,holdout
84266,tcga,KEAP1,1,45b9b476-a25a-4d35-8993-efabde165a11_13da6562-...,-2.448507,0.0,holdout
84267,tcga,KEAP1,1,5421f004-9b7f-47df-9849-779c274e9c48_2cbc7636-...,-1.844918,0.0,holdout
84268,tcga,KEAP1,1,5b2b09fa-e002-43a2-a4f8-e104c126d6d0_e07f3014-...,-1.831375,0.0,holdout


In [368]:
(ensemble
 .groupby(["split", "marker", "test_fold", "train_cohort"])
 .apply(evaluate_metrics)
 .groupby(["split", "marker", "train_cohort"])
 .mean()
 .round(2))

/var/folders/8x/9t509mc57cd14s_khxwbcpv40000gn/T/ipykernel_50755/1870056083.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(evaluate_metrics)


auc    f1  precision  recall  specificity  \
split   marker train_cohort                                               
holdout EGFR   hlcc          0.76  0.36       0.23    0.80         0.57   
               tcga          0.67  0.03       0.61    0.02         1.00   
        KEAP1  hlcc          0.47  0.01       0.08    0.01         0.99   
               tcga          0.51  0.02       0.01    0.03         0.97   
        KRAS   hlcc          0.56  0.46       0.33    0.76         0.25   
               tcga          0.52  0.33       0.38    0.34         0.68   
        STK11  hlcc          0.71  0.35       0.31    0.42         0.84   
               tcga          0.61  0.13       0.08    0.47         0.65   
        TP53   hlcc          0.75  0.68       0.70    0.66         0.68   
               tcga          0.64  0.49       0.59    0.44         0.73   
test    EGFR   hlcc          0.76  0.44       0.36    0.58         0.78   
               tcga          0.74  0.29       0.39    0.26         0.95   
        KEAP1  hlcc          0.66  0.09       0.08    0.10         0.97   
               tcga          0.58  0.17       0.34    0.15         0.95   
        KRAS   hlcc          0.62  0.48       0.45    0.53         0.65   
               tcga          0.69  0.50       0.54    0.48         0.80   
        STK11  hlcc          0.71  0.23       0.20    0.28         0.93   
               tcga          0.78  0.26       0.33    0.23         0.93   
        TP53   hlcc          0.70  0.62       0.60    0.66         0.61   
               tcga          0.74  0.71       0.69    0.74         0.62   

                             sensitivity  
split   marker train_cohort               
holdout EGFR   hlcc                 0.80  
               tcga                 0.02  
        KEAP1  hlcc                 0.01  
               tcga                 0.03  
        KRAS   hlcc                 0.76  
               tcga                 0.34  
        STK11  hlcc                 0.42  
               tcga                 0.47  
        TP53   hlcc                 0.66  
               tcga                 0.44  
test    EGFR   hlcc                 0.58  
               tcga                 0.26  
        KEAP1  hlcc                 0.10  
               tcga                 0.15  
        KRAS   hlcc                 0.53  
               tcga                 0.48  
        STK11  hlcc                 0.28  
               tcga                 0.23  
        TP53   hlcc                 0.66  
               tcga                 0.74

In [369]:
single = duckdb.sql("""
select 
    train_cohort,
    marker, 
    test_fold,
    dev_fold,
    seed,
    wsi_uuid,
    prediction, 
    label, 
    split
from prediction p 

join experiments e on e.experiment_uuid = p.experiment_uuid
where p.split in ('holdout', 'test')
""").df()

gcol = [
    "train_cohort",
    "test_fold", 
    "dev_fold",
    "seed", 
    "split",
    "marker",
]
(single
 .groupby(gcol)
 .apply(evaluate_metrics)
 .groupby(["split", "marker", "train_cohort"])
 .mean()
 .round(3))

/var/folders/8x/9t509mc57cd14s_khxwbcpv40000gn/T/ipykernel_50755/2732082443.py:56: RuntimeWarning: invalid value encountered in scalar divide
  precision = tp / (tp+fp) # PPV
/var/folders/8x/9t509mc57cd14s_khxwbcpv40000gn/T/ipykernel_50755/2558348954.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(evaluate_metrics)


auc     f1  precision  recall  specificity  \
split   marker train_cohort                                                 
holdout EGFR   hlcc          0.732  0.344      0.227   0.748        0.573   
               tcga          0.611  0.095      0.446   0.066        0.972   
        KEAP1  hlcc          0.474  0.045      0.099   0.037        0.954   
               tcga          0.510  0.036      0.027   0.086        0.921   
        KRAS   hlcc          0.541  0.425      0.342   0.664        0.355   
               tcga          0.517  0.331      0.373   0.392        0.631   
        STK11  hlcc          0.680  0.314      0.271   0.420        0.799   
               tcga          0.589  0.129      0.082   0.454        0.659   
        TP53   hlcc          0.697  0.634      0.672   0.641        0.619   
               tcga          0.605  0.481      0.562   0.477        0.650   
test    EGFR   hlcc          0.723  0.405      0.323   0.587        0.719   
               tcga          0.679  0.290      0.326   0.309        0.902   
        KEAP1  hlcc          0.610  0.083      0.060   0.189        0.914   
               tcga          0.559  0.170      0.194   0.189        0.892   
        KRAS   hlcc          0.568  0.405      0.391   0.513        0.566   
               tcga          0.659  0.456      0.462   0.477        0.718   
        STK11  hlcc          0.660  0.163      0.114   0.331        0.835   
               tcga          0.726  0.313      0.336   0.330        0.891   
        TP53   hlcc          0.645  0.578      0.576   0.625        0.574   
               tcga          0.699  0.663      0.666   0.673        0.616   

                             sensitivity  
split   marker train_cohort               
holdout EGFR   hlcc                0.748  
               tcga                0.066  
        KEAP1  hlcc                0.037  
               tcga                0.086  
        KRAS   hlcc                0.664  
               tcga                0.392  
        STK11  hlcc                0.420  
               tcga                0.454  
        TP53   hlcc                0.641  
               tcga                0.477  
test    EGFR   hlcc                0.587  
               tcga                0.309  
        KEAP1  hlcc                0.189  
               tcga                0.189  
        KRAS   hlcc                0.513  
               tcga                0.477  
        STK11  hlcc                0.331  
               tcga                0.330  
        TP53   hlcc                0.625  
               tcga                0.673

In [402]:
def selective_prediction_metrics(select_fraction=50):
    def wrapped(df):
        lo, hi = np.percentile(df.prediction, [select_fraction/2, 100-select_fraction/2])
        sub = df[df.prediction.ge(hi) | df.prediction.le(lo)]
        return pd.Series(calculate_metrics_at_threshold(0)(sub))
    return wrapped

tmp = ensemble.groupby(["train_cohort", "marker", "test_fold", "split"]).apply(selective_prediction_metrics(50))
tmp = tmp.groupby(["split", "marker", "train_cohort"]).mean().round(2)
tmp = tmp.reorder_levels(["split", "train_cohort", "marker"], 0)
tmp = tmp.sort_index()
tmp.to_clipboard()

/var/folders/8x/9t509mc57cd14s_khxwbcpv40000gn/T/ipykernel_50755/2158270851.py:8: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  tmp = ensemble.groupby(["train_cohort", "marker", "test_fold", "split"]).apply(selective_prediction_metrics(50))


In [358]:
tmp.loc['holdout'].query("train_cohort=='hlcc'")

,,auc,f1,precision,recall,specificity,sensitivity
marker,train_cohort,,,,,,
EGFR,hlcc,0.76,0.36,0.23,0.80,0.57,0.80
KEAP1,hlcc,0.47,0.01,0.08,0.01,0.99,0.01
KRAS,hlcc,0.56,0.46,0.33,0.76,0.25,0.76
STK11,hlcc,0.71,0.35,0.31,0.42,0.84,0.42
TP53,hlcc,0.75,0.68,0.70,0.66,0.68,0.66


In [384]:
for split in ["holdout", "test"]:
    for train_cohort in ["hlcc", "tcga"]:
        out = []
        ratios = list(reversed([20, 30, 40, 50, 60, 70, 80, 90, 100]))
        for thresh in ratios:
            score = (ensemble
             .query(f"split=='{split}'")
             .query(f"train_cohort=='{train_cohort}'")
             .groupby(["train_cohort", "marker", "test_fold", "split"])
             .apply(selective_prediction_metrics(thresh), include_groups=False)
             .groupby("marker")
             .mean()
        )
            out.append(score)
        out = pd.concat(out, keys=ratios)
        auc = out['auc'].unstack().melt(ignore_index=False, value_name="AUC").reset_index(names=["% most certain cases"])

        fig, ax = plt.subplots(1, 1, figsize=(4.1, 2.9))
        sns.lineplot(
            auc, 
            x='% most certain cases', 
            y='AUC', 
            hue='marker',
            linewidth=2,
            ax=ax
        )
        #ax.set_ylim(0.5, 0.95)
        #ax.invert_xaxis()
        #sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

        out = []
        for thresh in ratios:
            score = (single
             .query(f"split=='{split}'")
             .query(f"train_cohort=='{train_cohort}'")
             .groupby(["train_cohort", "marker", "test_fold", "split"])
             .apply(selective_prediction_metrics(thresh), include_groups=False)
             .groupby("marker")
             .mean()
        )
            out.append(score)

        out = pd.concat(out, keys=ratios)
        auc = out['auc'].unstack().melt(ignore_index=False, value_name="AUC").reset_index(names=["% most certain cases"])
        sns.lineplot(
            auc, 
            x='% most certain cases', 
            y='AUC', 
            hue='marker',
            linewidth=2,
            ax=ax,
            legend=False,
            linestyle=":",
            alpha=0.3,
        )
        ax.set_ylim(0.5, 0.95)
        ax.invert_xaxis()
        sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
        
        fig.savefig(f"{split}_{train_cohort}.png", dpi=600, bbox_inches='tight')
        fig.clf(); fig.clear()

<Figure size 410x290 with 0 Axes>

<Figure size 410x290 with 0 Axes>

<Figure size 410x290 with 0 Axes>

<Figure size 410x290 with 0 Axes>